# Week 4 — Applied SaaS Notebook

This notebook is part of the 'Applied ML Foundations for SaaS Analytics' course. Conversational, mentor-style guidance is provided throughout.

In [ ]:
from IPython.display import HTML
HTML('''
<style>
details {
  margin: 10px 0;
  padding: 8px 12px;
  border: 1px solid #d9e2ec;
  border-radius: 8px;
  background: #f9fbfd;
}
details summary {
  font-weight: 600;
  color: #0056b3;
  cursor: pointer;
}
details[open] {
  background: #f1f7ff;
  border-color: #c3d4f0;
}
details pre {
  background: #f8f9fa;
  padding: 8px;
  border-radius: 6px;
}
</style>
''')

## Scenario — CFO asks for churn by plan and region

We will clean subscriptions and events data, compute churn cohorts, and produce a weekly MRR lost report.


## Hands-on

Build a cohort table (signup_month -> retention at 30/60/90 days) using merge with events.


<details>
<summary>💡 Hint</summary>

Try breaking the problem into smaller steps. For example, if you need to aggregate per-user metrics, first compute a grouped table, then convert to NumPy arrays for vectorized ops. Think about edge cases: missing users, zero counts, or extreme values.

</details>

<details>
<summary>✅ Solution (example)</summary>

```python
# Example solution snippet — adapt to your dataset & question.
import pandas as pd
import numpy as np

# Load data (adjust path as needed)
df = pd.read_csv('../data/feature_usage.csv', parse_dates=['date'], low_memory=False)

# Example: compute total usage per user and return top users
user_usage = df.groupby('user_id')['usage_count'].sum().reset_index(name='total_usage')
top_users = user_usage.sort_values('total_usage', ascending=False).head(10)
top_users
```

**Why this works:** We use `groupby` to aggregate events by `user_id`, then sort to find the heaviest users. Converting to NumPy arrays can speed up numeric-only operations.

</details>

In [ ]:

import pandas as pd
subs = pd.read_csv('../data/subscriptions.csv', parse_dates=['signup_date','churn_date'], low_memory=False)
print('subscriptions:', subs.shape)
subs['signup_month'] = subs['signup_date'].dt.to_period('M')
# simple churn rate by plan
churn_by_plan = subs.groupby('plan_type')['is_churned'].mean().reset_index(name='churn_rate')
print(churn_by_plan)


## Reflection

If churn is higher in a region, what product or pricing hypotheses would you investigate?
